# Dependencies

In [1]:
import pandas as pd
import numpy as np
from constraint import *
import random
from scipy import optimize as spopt

In [2]:
DISTANCE_DATASET_PATH = "distances.xlsx"
INDEX_VALUES_DATASET_PATH = "index_values.csv"
INITIAL_DISTRIBUTION_DATASET_PATH = "initial_distribution.txt"
N_SELLERS = 4
N_BRICKS = 22

# Dataset loader

In [3]:
class Dataset:
    def __init__(self, distance_dataset_path, index_values_dataset_path, initial_distribution_dataset_path):
        # Store dataset path
        self.distance_dataset_path = distance_dataset_path
        self.index_values_dataset_path = index_values_dataset_path
        self.initial_distribution_dataset_path = initial_distribution_dataset_path
        
        # Read dataset
        self.read_distances()
        self.read_index_values()
        self.read_initial_distribution()
    
    def read_distances(self):
        """
        Load and store the data from the excel sheet provided in the object
        """
        # Load first sheet 
        brick_bureau = pd.read_excel(self.distance_dataset_path, skiprows=[0], sheet_name=0, header=0, index_col=1, dtype=str)
        # Drop empty cells of the first column
        brick_bureau = brick_bureau.drop(labels=brick_bureau.columns[0], axis=1)
        
        # Load second sheet
        brick_brick = pd.read_excel(self.distance_dataset_path, sheet_name=1, header=1, index_col=1, dtype=str)
        # Drop emty cells of the first column
        brick_brick = brick_brick.drop(labels=brick_brick.columns[0], axis=1)
        
        # Store in self
        self.brick_bureau = brick_bureau
        self.brick_brick = brick_brick
        
        
    def read_index_values(self):
        """ Read index values, les_v
        """
        self.index_values = np.genfromtxt(self.index_values_dataset_path, delimiter=',')
    
    def read_initial_distribution(self):
        """ Read initial state of bricks distribution betweek sellers: les_a
        """
        with open(self.initial_distribution_dataset_path, "r") as file:
            raw_data = file.read().split("\n")
            # remove header
            raw_data.pop(0)
            initial_distribution = []
            for line in raw_data:
                line.replace(" ", "")
#                 id_seller = line.split(":")[0]
                les_id_bricks = line.split(":")[1].split(",")
                initial_distribution.append(les_id_bricks)
        
        les_a = np.zeros((N_SELLERS, N_BRICKS))
        for id_seller in range(len(initial_distribution)):
            for id_brick in initial_distribution[id_seller]:
                les_a[id_seller, int(id_brick)-1] = 1
        
        self.initial_affectation = les_a
        
    
    def disp_dataset(self):
        for dataset in [self.brick_brick, self.brick_brick, self.index_values, self.initial_affectation]:
            print("")
            print("="*20)
            print(dataset)
    
        
        
dataset = Dataset(DISTANCE_DATASET_PATH, INDEX_VALUES_DATASET_PATH, INITIAL_DISTRIBUTION_DATASET_PATH)

In [4]:
# dataset.disp_dataset()

# Problem definition

### VARIABLES
- $x_{ij}$ ; = 1 if seller `i` is allocated to brick `j`, 0 otherwise.  

### PARAMETERS
- $d_{ij}$ ; distance from office of seller `i` to the brick `j`  
- $v_j$ ; index value of the `j` brick
- $a_{ij}$ ; = 1 if seller `i` was originally allocated to brick `j`, 0 otherwise.  


## Minimize total distance traveled

We want to minimize the total traveled distance, then we want to minimize:  
$\underset{i=1}{\overset{4}{\sum}}\underset{j=1}{\overset{22}{\sum}}x_{ij}d_{ij}$  

## Minimize imbalance in workload

We want to minimize the maximum of difference between the workers. They have to share 1/4 of the sum of the index values of bricks. Then, we want to minimize the absolute difference between this value and the seller workload.  
This way, we want to minimize:  
$\underset{i\in\{1, 2, 3, 4\}}{max}\left(\frac{\underset{j=1}{\overset{22}{\sum}} x_{ij}v_j}{\frac{1}{4} \underset{j=1}{\overset{22}{\sum}} v_j}\right)$

## Minimize distruption

We want to minimize the changes from the previous model to the new one, leading to minimizing:  
$\underset{i=1}{\overset{4}{\sum}}\underset{j=1}{\overset{22}{\sum}}x_{ij}(1-a_{ij})v_j$

- x: Matrix of shape number_of_seller=4 * number_of_bricks=22 with 0 or 1
- a: Same shape than x
- d: Matrix of shape number_of_seller=4 * number_of_bricks=22 with float

## Domain

$\forall i \in [1, 4], \forall j \in [1, 22], x_{ij} \in \{0, 1\}$

## Constraints

- 1 brick affectation to 1 seller  
$\rightarrow \forall j \in [0, 22], \underset{i=1}{\overset{4}{\sum}}x_{ij} = 1$
- Work imbalance between 80% and 120%:  
$\rightarrow$ work imbalance is set as a constraint. `const_imbalance_worload(les_x)` $\in [0.8, 1.2]$


## Tools

In [5]:
class Tools:
    @staticmethod
    def get_total_workload(les_v):
        return np.sum(les_v)
    
    @staticmethod
    def flatten(les_x):
        """ Flatten a numpy array"""
        return les_x.reshape(-1)
    
    @staticmethod
    def reshape(les_x):
        """ Reshape a numpy array to the shape (N_SELLERS, N_BRICKS)"""
        
        if les_x.shape != (N_SELLERS, N_BRICKS):
            les_x = les_x.reshape((N_SELLERS, N_BRICKS))
        return les_x
    
    

## Costs

In [6]:
class Cost:
    
    def __init__(self, les_d, les_v, les_a):
        self.les_d = les_d
        self.les_v = les_v
        self.les_a = les_a
        self.total_workload = Tools.get_total_workload(self.les_v)
        
        
    def cost_distance(self, les_x):
        """ Return cost regarding distances
        :param les_x: matrix numpy, with x[i, j] set to 1 if seller i have to cover brick j
        
        :return cost: cost value regarding distances
        """
        return np.sum(les_x * self.les_d)
    
    
    def cost_imbalance_worload(self, les_x):
        """ Return cost regarding imbalance workload
        :param les_x: matrix numpy, with x[i, j] set to 1 if seller i have to cover brick j
        
        :return cost: cost value regarding imbalance workloard
        """
        
        les_workload = [np.abs(0.25*self.total_workload - np.sum(les_x_seller * self.les_v)) for les_x_seller in les_x]
        return np.max(les_workload)
    
    
    def cost_disruption(self, les_x):
        """ Return cost regarding disruption"""
        return np.sum(les_x * ( 1 - self.les_a )*self.les_v )
    
    
    def cost_weight_distance_disruption(self, les_x, alpha=0.5):
        """Weighted cost function to minimize distance and disruption with provided alpha
        :param les_x: matrix numpy, with x[i, j] set to 1 if seller i have to cover brick j
        :param alpha: float, default=0.5, weight of the distance cost function (1-alpha is given to disruption)
        
        :return cost: weighted cost value with distances and disruption
        """
        
        les_x = Tools.reshape(les_x)
        
        return alpha*self.cost_distance(les_x) + (1-alpha)*self.cost_disruption(les_x)
    
    

## Constraints

In [7]:
class Constraint:
    
    def __init__(self, les_d, les_v, les_a):
        self.les_d = les_d
        self.les_v = les_v
        self.les_a = les_a
        self.total_workload = Tools.get_total_workload(self.les_v)
    
    
    def const_imbalance_workload(self, les_x):
        les_workload = [np.sum(les_x_seller * self.les_v)/(0.25*self.total_workload) for les_x_seller in les_x]
        return np.array(les_workload)
    
    
    def const_unity_allocation(self, les_x, brick_index):
        return np.sum(les_x[:,brick_index])
    
        

## Optimizer

In [17]:
class Optimizer:
    def __init__(self, coster, constrainer):
        self.coster = coster
        self.constrainer = constrainer
    
    
    def const_1(self, les_x):
        """ Constrain regarding 'every brick is allocated to 1 seller' """
        
        les_x = Tools.reshape(les_x)
        
        les_unity_allocation = []
        for j in range(N_SELLERS):
            les_unity_allocation.append(self.constrainer.const_unity_allocation(les_x, j) - 1)
        return np.sum(les_unity_allocation)
    
    
    def const_2(self, les_x):
        """ Constrain regarding worked is not working less than 80% of average """
        
        les_x = Tools.reshape(les_x)
        
        return self.constrainer.const_imbalance_workload(les_x) - 0.8
    
    
    def const_3(self, les_x):
        """ Constrain regarding worked is not working more than 120% of average """
        
        les_x = Tools.reshape(les_x)
        
        return 1.2 - self.constrainer.const_imbalance_workload(les_x)
    
    def const_4(self, les_x):
        """ Ensure the sum of les x is equal to the number of brick"""
        return np.sum(les_x) - N_BRICKS
    
    def const_5(self, les_x):
        """ Ensure max is 1"""
        return np.max(les_x) - 1
    
    def const_6(self, les_x):
        """ Ensure min is 0"""
        return np.min(les_x)
    
    
    def minimize(self, fun, les_x0):
        
        # Let's define constraints. For eq, have to be equal to 0. For ineq, have to be positive.
        cons = [{'type':'eq', 'fun': self.const_1},
                {'type':'ineq', 'fun': self.const_2},
                {'type':'ineq', 'fun': self.const_3},
                {'type':'eq', 'fun': self.const_4},
                {'type':'eq', 'fun': self.const_6}]
        
        return spopt.minimize(fun, les_x0, constraints=cons, options={'disp': True})
        
        

In [22]:
class Interface:
    
    def __init__(self):
        self.dataset = Dataset(DISTANCE_DATASET_PATH, INDEX_VALUES_DATASET_PATH, INITIAL_DISTRIBUTION_DATASET_PATH)
        self.les_d = self.get_les_d()
        self.les_v = self.get_les_v()
        self.les_a = self.get_les_a()
        self.les_x_test = self.generate_les_x_test()
        
        self.coster = Cost(les_d=self.les_d, les_v=self.les_v, les_a=self.les_a)
        
        self.constrainer = Constraint(les_d=self.les_d, les_v=self.les_v, les_a=self.les_a)
        
        self.optimizer = Optimizer(self.coster, self.constrainer)

    
    def get_les_d(self):
        """ Return value of distances betweek sellers office and bricks
        """
        les_d = np.array(self.dataset.brick_bureau, dtype=np.float).transpose()
        return les_d
    
    
    def get_les_v(self):
        """ Return index values of brickes, with les_v[j] = index value of brick j
        """
        les_v = self.dataset.index_values.reshape((1, -1))
        return les_v

    
    def get_les_a(self):
        """ Return les a"""
        les_a = self.dataset.initial_affectation
        return les_a
    
    
    def generate_les_x_test(self):
        """
        To test cost function, we need to generate a false affectation of seller to bricks.
        Thus, we create a random affectation.
        """
        # Create a x_test matrix, inversed shape, with line corresponding to bricks, columns to sellers
        les_x_test = np.zeros((N_BRICKS, N_SELLERS))
        
        # Put one 1 per line, to affect each brick to a seller
        for id_line in range(len(les_x_test)):
            id_one = random.randint(0, len(les_x_test[id_line]) - 1)
            les_x_test[id_line, id_one] = 1
        
        # Lets transpose the matrix, as we want seller as rows.
        les_x_test = les_x_test.transpose()
        return les_x_test
    
    
    def test_cost_distance(self):
        print("Test cost distance")
        print(self.coster.cost_distance(les_x=self.les_x_test))
        
        
    def test_cost_imbalance_worload(self):
        print("Test cost imbalance")
        print(self.coster.cost_imbalance_worload(les_x=self.les_x_test))
        
    def test_cost_distruption(self):
        print("Test cost disruption")
        print(self.coster.cost_disruption(les_x=self.les_x_test))
        
    def test_const_imbalance_workload(self):
        print("Test const imbalance workload")
        print(self.constrainer.const_imbalance_workload(les_x=self.les_x_test))
        
    def test_const_unity_allocation(self):
        les_unit = []
        for j in range(N_BRICKS):
            les_unit.append(self.constrainer.const_unity_allocation(les_x=self.les_x_test, brick_index=j))
        
        print(les_unit)
        
    
    def optimize(self):
        fun = self.coster.cost_weight_distance_disruption
        les_x0 = self.les_x_test
        res = self.optimizer.minimize(fun, les_x0)
        self.res = res
    
    def show_results(self):
        print("random x")
        print(self.les_x_test)
        print("Computed result")
        resx = Tools.reshape(self.res.x)
        print(resx)
        
        
            
    
interfacer = Interface()
# interfacer.test_cost_distance()
# interfacer.test_cost_imbalance_worload()
# interfacer.test_cost_distruption()

# interfacer.test_const_imbalance_workload()
# interfacer.test_const_unity_allocation()

interfacer.optimize()
interfacer.show_results()

Singular matrix C in LSQ subproblem    (Exit mode 6)
            Current function value: 186.70695000000003
            Iterations: 1
            Function evaluations: 90
            Gradient evaluations: 1
random x
[[0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
Computed result
[[0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


In [19]:
print(x_result)


22.0
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


In [ ]:
np.s